# NLTK Complete Guide - Section 16: Advanced Topics

## Overview

This notebook covers advanced NLP concepts that go beyond basic text processing. These techniques are essential for building sophisticated NLP applications.

### Topics Covered

| Topic | Description | Use Cases |
|-------|-------------|-----------|
| **Context-Free Grammar** | Formal rules for sentence structure | Parsing, grammar checking |
| **Probabilistic CFG** | Grammar with rule probabilities | Disambiguation, language generation |
| **Information Extraction** | Extract structured data from text | Emails, phones, dates from text |
| **Relation Extraction** | Find relationships between entities | Knowledge graphs, QA systems |
| **Text Normalization** | Standardize text preprocessing | Pipeline building, data cleaning |
| **Performance Optimization** | Speed up NLP operations | Production systems, large datasets |

### Prerequisites

This section assumes familiarity with:
- Tokenization and POS tagging
- Basic Python classes and functions
- Regular expressions basics

In [ ]:
import nltk
import re

nltk.download('punkt', quiet=True)
nltk.download('averaged_perceptron_tagger', quiet=True)
nltk.download('maxent_ne_chunker', quiet=True)
nltk.download('words', quiet=True)

from nltk import CFG, ChartParser, RecursiveDescentParser
from nltk import pos_tag, word_tokenize, ne_chunk
from nltk.chunk import RegexpParser
from nltk.tree import Tree

### Understanding the Imports

| Import | Purpose |
|--------|---------|
| `CFG` | Define Context-Free Grammar rules |
| `ChartParser` | Efficient parsing algorithm |
| `RecursiveDescentParser` | Top-down parsing (simpler) |
| `RegexpParser` | Pattern-based chunking |
| `Tree` | Tree data structure for parse trees |
| `ne_chunk` | Named Entity Recognition chunking |

## 16.1 Context-Free Grammar (CFG)

### What is a Grammar?

A **grammar** defines the rules for constructing valid sentences in a language. In NLP, we use **Context-Free Grammar (CFG)** to describe sentence structure.

### CFG Components

| Component | Symbol | Example | Description |
|-----------|--------|---------|-------------|
| **Non-terminal** | S, NP, VP | S, NP | Abstract categories |
| **Terminal** | lowercase | 'dog', 'the' | Actual words |
| **Production Rule** | → | S → NP VP | How to expand non-terminals |
| **Start Symbol** | S | S | Where parsing begins |

### Grammar Notation

```
S  → NP VP      # A sentence is a noun phrase + verb phrase
NP → Det N      # A noun phrase is a determiner + noun
VP → V NP       # A verb phrase is a verb + noun phrase
Det → 'the'     # Terminal symbols are actual words
N → 'dog'
V → 'chased'
```

### Why CFG Matters

- **Parsing**: Understand sentence structure
- **Grammar checking**: Validate sentence correctness
- **Language generation**: Generate valid sentences
- **Machine translation**: Transfer structure between languages

In [ ]:
# Define a simple Context-Free Grammar
# CFG.fromstring() parses a multi-line grammar specification

grammar = CFG.fromstring("""
    S -> NP VP
    NP -> Det N | Det Adj N | 'I'
    VP -> V NP | V
    Det -> 'the' | 'a'
    N -> 'dog' | 'cat' | 'ball' | 'park'
    Adj -> 'big' | 'small' | 'happy'
    V -> 'chased' | 'saw' | 'ran'
""")

print("CONTEXT-FREE GRAMMAR")
print("=" * 55)
print("""
This grammar defines simple English sentences like:
  "the dog chased a cat"
  "I saw the big ball"

Grammar Components:
  S   = Sentence (start symbol)
  NP  = Noun Phrase
  VP  = Verb Phrase  
  Det = Determiner (the, a)
  N   = Noun
  Adj = Adjective
  V   = Verb
""")

print("Production Rules:")
print("-" * 55)
for production in grammar.productions():
    print(f"  {production}")

In [ ]:
# Parse a sentence using the grammar
# ChartParser efficiently finds all valid parse trees

parser = ChartParser(grammar)

sentence = "the big dog chased a cat".split()

print("PARSING A SENTENCE")
print("=" * 55)
print(f"\nSentence: \"{' '.join(sentence)}\"")
print(f"Tokens:   {sentence}")
print("\n" + "-" * 55)
print("Parse Tree(s):")

for i, tree in enumerate(parser.parse(sentence), 1):
    print(f"\nInterpretation {i}:")
    print(tree)
    print("\nVisualization:")
    tree.pretty_print()

print("""
💡 Reading the Parse Tree:
   • S at root = valid sentence
   • Each level shows phrase structure
   • Leaves are the actual words
""")

In [ ]:
# Demonstrating AMBIGUITY in grammar
# Some sentences have multiple valid interpretations!

complex_grammar = CFG.fromstring("""
    S -> NP VP
    NP -> Det N | Det N PP | 'I' | N
    VP -> V | V NP | V NP PP
    PP -> P NP
    Det -> 'the' | 'a' | 'my'
    N -> 'dog' | 'cat' | 'park' | 'telescope' | 'man' | 'hill'
    V -> 'saw' | 'walked' | 'chased'
    P -> 'in' | 'on' | 'with' | 'by'
""")

parser = ChartParser(complex_grammar)

# This sentence is AMBIGUOUS!
sentence = "I saw the man with the telescope".lower().split()

print("SYNTACTIC AMBIGUITY")
print("=" * 60)
print(f"\nSentence: \"{' '.join(sentence)}\"")
print("""
This sentence has TWO valid interpretations:

1. I used a telescope to see the man
   "with the telescope" modifies the VERB (how I saw)

2. I saw a man who had a telescope  
   "with the telescope" modifies the NOUN (which man)
""")
print("-" * 60)

for i, tree in enumerate(parser.parse(sentence), 1):
    print(f"\n🔍 Interpretation {i}:")
    tree.pretty_print()
    
    # Explain the interpretation
    if i == 1:
        print("   → PP 'with telescope' attached to VP (I used the telescope)")
    else:
        print("   → PP 'with telescope' attached to NP (the man has telescope)")

print("""
💡 Ambiguity is a major challenge in NLP!
   Context and world knowledge help humans disambiguate,
   but machines need additional techniques (PCFGs, statistics).
""")

## 16.2 Probabilistic CFG (PCFG)

### The Problem with Plain CFG

Regular CFG can find ALL valid parses, but:
- Multiple parses may exist (ambiguity)
- All parses are treated equally
- No way to prefer more likely interpretations

### Solution: Add Probabilities

**Probabilistic CFG (PCFG)** adds probability to each rule:
- Rules for the same non-terminal must sum to 1.0
- Parse probability = product of all rule probabilities
- Choose the most probable parse

```
# Regular CFG               # PCFG with probabilities
NP -> Det N                 NP -> Det N [0.6]
NP -> Det Adj N             NP -> Det Adj N [0.4]
```

### Why PCFG?

| Benefit | Description |
|---------|-------------|
| **Disambiguation** | Prefer common constructions |
| **Ranking** | Sort parses by probability |
| **Generation** | Generate natural-sounding text |

In [ ]:
from nltk import PCFG, ViterbiParser

# Define a Probabilistic CFG
# Each rule has a probability in brackets [p]
# Rules for same LHS must sum to 1.0

pcfg = PCFG.fromstring("""
    S -> NP VP [1.0]
    NP -> Det N [0.5] | Det Adj N [0.3] | 'I' [0.2]
    VP -> V NP [0.7] | V [0.3]
    Det -> 'the' [0.6] | 'a' [0.4]
    N -> 'dog' [0.4] | 'cat' [0.3] | 'ball' [0.3]
    Adj -> 'big' [0.5] | 'small' [0.5]
    V -> 'chased' [0.5] | 'saw' [0.5]
""")

print("PROBABILISTIC CFG")
print("=" * 60)
print("""
Each rule now has an associated probability:
  • NP → Det N [0.5] means 50% of NPs follow this pattern
  • Det → 'the' [0.6] means 'the' is more common than 'a'
""")
print("Sample Productions:")
print("-" * 60)

for prod in pcfg.productions()[:10]:
    print(f"  {prod}")

In [ ]:
# ViterbiParser finds the MOST PROBABLE parse tree
# (Uses dynamic programming, like in speech recognition)

viterbi_parser = ViterbiParser(pcfg)

sentence = "the dog chased a cat".split()

print("VITERBI PARSING (Finding Most Probable Parse)")
print("=" * 60)
print(f"\nSentence: \"{' '.join(sentence)}\"")
print("-" * 60)

for tree in viterbi_parser.parse(sentence):
    prob = tree.prob()
    print(f"\n📊 Parse Probability: {prob:.8f}")
    print(f"\nHow it's calculated:")
    print(f"  P(S→NP VP)   × P(NP→Det N)  × P(Det→'the') × ...")
    print(f"  = 1.0        × 0.5          × 0.6          × ...")
    print(f"  = {prob:.8f}")
    print(f"\nParse Tree:")
    tree.pretty_print()

print("""
💡 The Viterbi algorithm efficiently finds the most probable
   parse without exploring all possibilities.
   
   This is essential for:
   • Disambiguating sentences
   • Speech recognition (choosing best interpretation)
   • Machine translation (selecting best structure)
""")

## 16.3 Information Extraction with Regular Expressions

**Information Extraction (IE)** finds structured data in unstructured text. The simplest approach uses **regular expressions** to match patterns.

### Common Extraction Targets

| Target | Pattern Example | Description |
|--------|-----------------|-------------|
| **Email** | `\w+@\w+\.\w+` | user@domain.com |
| **Phone** | `\d{3}-\d{3}-\d{4}` | 123-456-7890 |
| **URL** | `https?://[\w./]+` | http://example.com |
| **Date** | `\d{1,2}/\d{1,2}/\d{4}` | 12/25/2024 |
| **Price** | `\$[\d,]+\.?\d*` | $19.99 |
| **Hashtag** | `#\w+` | #python |

### Why Regex for IE?

✅ **Fast** - Compiled patterns are efficient  
✅ **Precise** - Exact pattern matching  
✅ **Portable** - Works across languages  
❌ **Limited** - Can't handle complex semantics  
❌ **Brittle** - Small format changes break patterns

In [ ]:
# Extract structured information using regular expressions

text = """
Contact us at support@example.com or sales@company.org.
Call 123-456-7890 or (555) 123-4567 for assistance.
Visit https://www.example.com or http://test.org for more info.
Prices: $19.99, $150, $1,299.00 - special offer ends 12/31/2024!
"""

print("INFORMATION EXTRACTION WITH REGEX")
print("=" * 60)
print(f"\nInput Text:\n{text}")
print("-" * 60)

# Define patterns with explanations
patterns = {
    'Emails': (r'[\w.-]+@[\w.-]+\.\w+', 
               'username @ domain . extension'),
    'Phones': (r'\(?\d{3}\)?[-.\s]?\d{3}[-.\s]?\d{4}', 
               'optional ( + 3 digits + optional ) + separator + 3 digits + separator + 4 digits'),
    'URLs': (r'https?://[\w./]+', 
             'http or https :// + domain'),
    'Prices': (r'\$[\d,]+\.?\d*', 
               '$ + digits with optional commas and decimals'),
    'Dates': (r'\d{1,2}/\d{1,2}/\d{2,4}', 
              'month/day/year format'),
}

print("\nExtracted Information:")
print("-" * 60)

for name, (pattern, explanation) in patterns.items():
    matches = re.findall(pattern, text)
    print(f"\n📌 {name}:")
    print(f"   Pattern: {pattern}")
    print(f"   Matches: {matches}")

In [ ]:
class PatternExtractor:
    """
    Reusable pattern extractor for common information types.
    
    This class provides a centralized way to extract various
    types of structured information from text.
    
    Example:
        results = PatternExtractor.extract_all(text)
        emails = PatternExtractor.extract(text, 'email')
    """
    
    # Pre-defined patterns for common information types
    patterns = {
        'email': r'[\w.-]+@[\w.-]+\.\w+',
        'phone': r'\(?\d{3}\)?[-.\s]?\d{3}[-.\s]?\d{4}',
        'url': r'https?://[\w./-]+',
        'price': r'\$[\d,]+\.?\d*',
        'date': r'\d{1,2}[/-]\d{1,2}[/-]\d{2,4}',
        'time': r'\d{1,2}:\d{2}(?:\s?[AP]M)?',
        'hashtag': r'#\w+',
        'mention': r'@\w+',
        'ip_address': r'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}',
        'percentage': r'\d+\.?\d*%',
    }
    
    @classmethod
    def extract(cls, text, pattern_name):
        """
        Extract specific pattern from text.
        
        Args:
            text: Input text
            pattern_name: Name of pattern (email, phone, etc.)
        
        Returns:
            List of matches
        """
        if pattern_name not in cls.patterns:
            raise ValueError(f"Unknown pattern: {pattern_name}. "
                           f"Available: {list(cls.patterns.keys())}")
        return re.findall(cls.patterns[pattern_name], text, re.IGNORECASE)
    
    @classmethod
    def extract_all(cls, text):
        """
        Extract ALL known patterns from text.
        
        Args:
            text: Input text
        
        Returns:
            Dictionary {pattern_name: [matches]}
        """
        results = {}
        for name, pattern in cls.patterns.items():
            matches = re.findall(pattern, text, re.IGNORECASE)
            if matches:
                results[name] = matches
        return results
    
    @classmethod
    def add_pattern(cls, name, pattern):
        """Add a custom pattern to the extractor."""
        cls.patterns[name] = pattern

print("✅ PatternExtractor class defined!")
print(f"\nAvailable patterns: {list(PatternExtractor.patterns.keys())}")

In [ ]:
# Demonstrate the PatternExtractor class
sample_text = """
📧 Meeting scheduled for 01/15/2024 at 2:30 PM.
Contact john.doe@email.com or call (555) 123-4567.
Check out our website: https://www.example.com/products
Follow us @company #innovation #tech #AI
Special offer: $99.99 (50% off!) Server IP: 192.168.1.100
"""

print("PATTERN EXTRACTOR DEMO")
print("=" * 60)
print(f"\nInput Text:{sample_text}")
print("-" * 60)

# Extract all patterns at once
results = PatternExtractor.extract_all(sample_text)

print("\n📊 Extracted Information:")
for pattern_type, matches in results.items():
    emoji_map = {
        'email': '📧', 'phone': '📞', 'url': '🔗',
        'date': '📅', 'time': '⏰', 'price': '💰',
        'hashtag': '#️⃣', 'mention': '@', 'percentage': '%',
        'ip_address': '🖥️'
    }
    emoji = emoji_map.get(pattern_type, '•')
    print(f"\n{emoji} {pattern_type.upper()}:")
    for match in matches:
        print(f"   {match}")

## 16.4 Relation Extraction

**Relation Extraction** identifies relationships between entities in text.

### Examples

| Text | Subject | Relation | Object |
|------|---------|----------|--------|
| "Apple acquired Beats" | Apple | acquired | Beats |
| "Einstein was born in Germany" | Einstein | born_in | Germany |
| "Python was created by Guido" | Python | created_by | Guido |

### Approaches

| Method | Description | Complexity |
|--------|-------------|------------|
| **Pattern-based** | Regex/rules for specific relations | Simple |
| **Chunking** | NP-VP-NP sequences | Medium |
| **Dependency parsing** | Analyze grammatical relations | Complex |
| **Deep Learning** | Neural models (BERT, etc.) | Advanced |

### Simple Approach: Chunking

We can use chunking grammar to find:
1. Noun Phrases (potential entities)
2. Verb Phrases (potential relations)
3. Extract NP-VP-NP patterns as triples

In [ ]:
def extract_relations(text):
    """
    Extract subject-relation-object triples from text.
    
    Uses chunking to identify:
    1. Noun Phrases (NP) - potential subjects/objects
    2. Verb Phrases (VP) - potential relations
    3. NP-VP-NP patterns as relation triples
    
    Args:
        text: Input sentence
    
    Returns:
        List of relation tuples
    """
    # Step 1: Tokenize and POS tag
    tokens = word_tokenize(text)
    tagged = pos_tag(tokens)
    
    # Step 2: Define chunking grammar
    # NP: Determiner? + Adjectives* + Nouns+
    # VP: Verb + optional Adverb
    grammar = r"""
        NP: {<DT>?<JJ>*<NN.*>+}
        VP: {<VB.*><RB>?}
        RELATION: {<NP><VP><NP>}
    """
    
    # Step 3: Parse with chunk grammar
    parser = RegexpParser(grammar)
    tree = parser.parse(tagged)
    
    # Step 4: Extract NP-VP-NP patterns
    relations = []
    
    for subtree in tree.subtrees():
        if subtree.label() == 'RELATION':
            parts = []
            for child in subtree:
                if isinstance(child, Tree):
                    # Combine words in the chunk
                    phrase = ' '.join(word for word, tag in child.leaves())
                    parts.append(phrase)
            if len(parts) >= 2:
                relations.append(tuple(parts))
    
    return relations

print("✅ extract_relations() function defined!")

In [ ]:
# Demonstrate relation extraction
sentences = [
    "The company acquired the startup.",
    "John founded a technology company.",
    "The scientists discovered a new species.",
    "Apple released the iPhone.",
    "The team won the championship.",
]

print("RELATION EXTRACTION")
print("=" * 60)

for sent in sentences:
    relations = extract_relations(sent)
    
    print(f"\n📝 Sentence: \"{sent}\"")
    
    if relations:
        for rel in relations:
            if len(rel) >= 3:
                subj, verb, obj = rel[0], rel[1], rel[2]
                print(f"   📌 Subject: {subj}")
                print(f"      Relation: {verb}")
                print(f"      Object: {obj}")
            elif len(rel) == 2:
                print(f"   📌 Partial: {rel}")
    else:
        print("   ⚠️ No clear NP-VP-NP pattern found")

print("""

💡 Limitations of Simple Relation Extraction:
   • Only captures basic Subject-Verb-Object patterns
   • Misses passive voice ("The startup was acquired")
   • Doesn't resolve pronouns ("It was founded by...")
   • For production, consider dependency parsing or ML models
""")

## 16.5 Text Normalization Pipeline

**Text normalization** standardizes text before analysis. A good pipeline handles multiple transformations consistently.

### Common Normalization Steps

| Step | Description | Example |
|------|-------------|---------|
| **Lowercase** | Convert to lowercase | "Hello" → "hello" |
| **Expand contractions** | "don't" → "do not" | Standardize forms |
| **Remove accents** | "café" → "cafe" | Handle special chars |
| **Remove punctuation** | "Hello!" → "Hello" | Clean tokens |
| **Remove stopwords** | Filter common words | Remove "the", "is" |
| **Lemmatization** | Reduce to base form | "running" → "run" |

### Why Normalization Matters

- **Consistency**: Same word, same representation
- **Reduce vocabulary**: "Running", "runs", "ran" → "run"
- **Better matching**: "café" matches "cafe"
- **Cleaner features**: Focus on meaningful content

In [ ]:
import unicodedata
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

nltk.download('stopwords', quiet=True)
nltk.download('wordnet', quiet=True)

class TextNormalizer:
    """
    Comprehensive text normalization pipeline.
    
    Combines multiple normalization techniques into a single,
    configurable pipeline for preprocessing text.
    
    Features:
    - Contraction expansion
    - Accent removal
    - Lowercasing
    - Punctuation removal
    - Stopword filtering
    - Lemmatization
    
    Example:
        normalizer = TextNormalizer()
        tokens = normalizer.normalize("I can't believe it's 2024!")
        # ['believe', '2024']
    """
    
    def __init__(self, language='english'):
        """Initialize with language-specific resources."""
        self.stop_words = set(stopwords.words(language))
        self.lemmatizer = WordNetLemmatizer()
        
        # Common English contractions
        self.contractions = {
            "won't": "will not", "can't": "cannot",
            "couldn't": "could not", "shouldn't": "should not",
            "wouldn't": "would not", "don't": "do not",
            "doesn't": "does not", "didn't": "did not",
            "haven't": "have not", "hasn't": "has not",
            "hadn't": "had not", "isn't": "is not",
            "aren't": "are not", "wasn't": "was not",
            "weren't": "were not", "let's": "let us",
            "n't": " not", "'re": " are",
            "'s": " is", "'d": " would",
            "'ll": " will", "'ve": " have",
            "'m": " am",
        }
        print("TextNormalizer initialized")
    
    def expand_contractions(self, text):
        """Expand contractions to full forms."""
        for contraction, expansion in self.contractions.items():
            text = text.replace(contraction, expansion)
        return text
    
    def remove_accents(self, text):
        """Remove accent marks from characters."""
        # Normalize to decomposed form, then remove combining characters
        nfkd = unicodedata.normalize('NFKD', text)
        return ''.join(c for c in nfkd if not unicodedata.combining(c))
    
    def normalize(self, text, 
                  lowercase=True,
                  remove_punctuation=True,
                  remove_numbers=False,
                  remove_stopwords=True,
                  lemmatize=True,
                  expand_contractions=True):
        """
        Full normalization pipeline.
        
        Args:
            text: Input text
            lowercase: Convert to lowercase
            remove_punctuation: Remove punctuation marks
            remove_numbers: Remove numeric tokens
            remove_stopwords: Filter out common words
            lemmatize: Reduce words to base form
            expand_contractions: Expand contractions
        
        Returns:
            List of normalized tokens
        """
        # Step 1: Expand contractions
        if expand_contractions:
            text = self.expand_contractions(text)
        
        # Step 2: Remove accents
        text = self.remove_accents(text)
        
        # Step 3: Lowercase
        if lowercase:
            text = text.lower()
        
        # Step 4: Tokenize
        tokens = word_tokenize(text)
        
        # Step 5: Filter tokens
        filtered = []
        for token in tokens:
            # Remove punctuation
            if remove_punctuation and not token.isalnum():
                continue
            
            # Remove numbers
            if remove_numbers and token.isdigit():
                continue
            
            # Remove stopwords
            if remove_stopwords and token.lower() in self.stop_words:
                continue
            
            # Lemmatize
            if lemmatize and token.isalpha():
                token = self.lemmatizer.lemmatize(token)
            
            filtered.append(token)
        
        return filtered
    
    def normalize_minimal(self, text):
        """Light normalization: lowercase and tokenize only."""
        return self.normalize(text, 
                             remove_punctuation=False,
                             remove_stopwords=False,
                             lemmatize=False)

In [ ]:
# Demonstrate the TextNormalizer
normalizer = TextNormalizer()

texts = [
    "I can't believe it's already 2024! The café was amazing.",
    "They're running 5 miles every day. She's been training hard.",
    "The dogs were happily playing with their toys in the gardens.",
]

print("TEXT NORMALIZATION EXAMPLES")
print("=" * 65)

for text in texts:
    normalized = normalizer.normalize(text)
    
    print(f"\n📝 Original:")
    print(f"   \"{text}\"")
    print(f"\n✨ Normalized:")
    print(f"   {normalized}")
    print(f"\n   Reduction: {len(text.split())} words → {len(normalized)} tokens")
    print("-" * 65)

# Show step-by-step normalization
print("\n📋 STEP-BY-STEP NORMALIZATION:")
print("-" * 65)
sample = "I can't believe it's AMAZING!"
print(f"Original: \"{sample}\"")
print(f"1. Expand contractions: \"{normalizer.expand_contractions(sample)}\"")
print(f"2. Remove accents: \"{normalizer.remove_accents(sample)}\"")
print(f"3. Full normalize: {normalizer.normalize(sample)}")

## 16.6 Performance Optimization

When processing large text collections, performance matters. Here are key optimization techniques.

### Optimization Strategies

| Technique | Description | Use Case |
|-----------|-------------|----------|
| **Caching** | Store repeated computations | Lemmatization, lookups |
| **Batch processing** | Process multiple items together | POS tagging sentences |
| **Compile patterns** | Pre-compile regex | Repeated matching |
| **Lazy loading** | Load resources only when needed | Large corpora |
| **Generators** | Process items one at a time | Memory efficiency |

### The `lru_cache` Decorator

Python's `functools.lru_cache` memoizes function results:
- First call: Computes and stores result
- Subsequent calls: Returns cached result
- Great for pure functions (same input → same output)

In [ ]:
import time
from functools import lru_cache

# OPTIMIZATION 1: Caching with lru_cache
# Perfect for repeated operations like lemmatization

@lru_cache(maxsize=10000)
def cached_lemmatize(word):
    """
    Cached lemmatization - stores results for repeated words.
    
    In real text, many words repeat (the, is, a, etc.)
    Caching avoids redundant computation.
    """
    lemmatizer = WordNetLemmatizer()
    return lemmatizer.lemmatize(word)

# Test performance: lemmatize 4000 words (with repetition)
words = ["running", "dogs", "happily", "better", "cats", "walked", "quickly", "good"] * 500

print("CACHING PERFORMANCE COMPARISON")
print("=" * 55)
print(f"Processing {len(words):,} words (repeated vocabulary)\n")

# Without cache
lemmatizer = WordNetLemmatizer()
start = time.time()
result1 = [lemmatizer.lemmatize(w) for w in words]
time_uncached = time.time() - start

# With cache (first run - building cache)
cached_lemmatize.cache_clear()  # Clear any existing cache
start = time.time()
result2 = [cached_lemmatize(w) for w in words]
time_first_cached = time.time() - start

# With cache (second run - using cache)
start = time.time()
result3 = [cached_lemmatize(w) for w in words]
time_cached = time.time() - start

print(f"{'Method':<25} {'Time':>10} {'Speedup':>10}")
print("-" * 55)
print(f"{'Without cache:':<25} {time_uncached:>10.4f}s {'-':>10}")
print(f"{'With cache (1st run):':<25} {time_first_cached:>10.4f}s {time_uncached/time_first_cached:>10.1f}x")
print(f"{'With cache (2nd run):':<25} {time_cached:>10.4f}s {time_uncached/time_cached:>10.1f}x")

# Cache statistics
info = cached_lemmatize.cache_info()
print(f"\n📊 Cache Statistics:")
print(f"   Hits: {info.hits:,} (returned from cache)")
print(f"   Misses: {info.misses:,} (computed fresh)")
print(f"   Hit ratio: {info.hits/(info.hits+info.misses)*100:.1f}%")

In [ ]:
# OPTIMIZATION 2: Batch Processing
# Many NLTK functions have batch versions that are more efficient

from nltk import pos_tag_sents

# Create test data: 300 sentences
sentences = [
    word_tokenize("The quick brown fox jumps over the lazy dog."),
    word_tokenize("Natural language processing is a fascinating field."),
    word_tokenize("Machine learning transforms industries worldwide."),
] * 100

print("BATCH PROCESSING COMPARISON")
print("=" * 55)
print(f"Processing {len(sentences)} sentences\n")

# Individual processing (one at a time)
start = time.time()
result1 = [pos_tag(sent) for sent in sentences]
time_individual = time.time() - start

# Batch processing (all at once)
start = time.time()
result2 = pos_tag_sents(sentences)
time_batch = time.time() - start

print(f"{'Method':<25} {'Time':>10} {'Speedup':>10}")
print("-" * 55)
print(f"{'Individual pos_tag():':<25} {time_individual:>10.4f}s {'-':>10}")
print(f"{'Batch pos_tag_sents():':<25} {time_batch:>10.4f}s {time_individual/time_batch:>10.1f}x")

print(f"""
💡 Why is batch processing faster?
   • Single model load (not repeated per sentence)
   • Optimized memory allocation
   • Better CPU cache utilization
   
📌 NLTK batch functions:
   • pos_tag_sents(sentences) - Tag multiple sentences
   • ne_chunk_sents(tagged) - NER on multiple sentences
""")

## Summary & Quick Reference

### Parsing

| Tool | Description | Use Case |
|------|-------------|----------|
| `CFG.fromstring()` | Define grammar from string | Create grammars |
| `ChartParser` | General-purpose parser | Find all parses |
| `ViterbiParser` | Find most probable parse | PCFG disambiguation |
| `RecursiveDescentParser` | Simple top-down parser | Teaching/debugging |

### Grammar Syntax

```python
from nltk import CFG, PCFG

# Context-Free Grammar
grammar = CFG.fromstring("""
    S -> NP VP
    NP -> Det N
    Det -> 'the' | 'a'
    N -> 'dog' | 'cat'
""")

# Probabilistic CFG (probabilities in brackets)
pcfg = PCFG.fromstring("""
    S -> NP VP [1.0]
    NP -> Det N [0.6] | N [0.4]
""")
```

### Information Extraction

```python
import re

# Common patterns
email = r'[\w.-]+@[\w.-]+\.\w+'
phone = r'\(?\d{3}\)?[-.\s]?\d{3}[-.\s]?\d{4}'
url = r'https?://[\w./]+'

# Extract
matches = re.findall(pattern, text)
```

### Text Normalization Checklist

1. ☐ Expand contractions ("don't" → "do not")
2. ☐ Remove accents ("café" → "cafe")
3. ☐ Lowercase
4. ☐ Tokenize
5. ☐ Remove punctuation
6. ☐ Remove stopwords
7. ☐ Lemmatize

### Performance Tips

| Technique | Code | Speedup |
|-----------|------|---------|
| **Cache results** | `@lru_cache(maxsize=N)` | 10-100x for repeated calls |
| **Batch processing** | `pos_tag_sents(list)` | 2-5x |
| **Compile regex** | `re.compile(pattern)` | 2-3x for repeated use |
| **Generator expressions** | `(x for x in items)` | Memory efficient |

### Key Takeaways

1. **CFG** formalizes sentence structure - essential for parsing
2. **PCFG** adds probabilities for disambiguation
3. **Regex** is fast for structured patterns (emails, phones)
4. **Normalization** is crucial for consistent preprocessing
5. **Optimization** matters at scale - use caching and batching

### Next Steps

- Build a complete NLP pipeline combining these techniques
- Explore dependency parsing for deeper analysis
- Try neural approaches (transformers, BERT) for production
- Section 17: Real-World Projects for practical applications